<a href="https://colab.research.google.com/github/coketaste/dl-examples/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install torch

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
torch.manual_seed(4242)

In [4]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist', train=True, download=True,
                                                          transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])),
                                           batch_size=64, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 111927848.48it/s]


Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 38370508.02it/s]


Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 60013462.43it/s]

Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 18040273.45it/s]


Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw



In [5]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)


In [6]:
model = Net()

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [8]:
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    print('Current loss', float(loss))

Current loss 0.48086777329444885
Current loss 0.13488271832466125
Current loss 0.44627058506011963
Current loss 0.4934777021408081
Current loss 0.2114088237285614
Current loss 0.28723108768463135
Current loss 0.23314730823040009
Current loss 0.2557545304298401
Current loss 0.03362549841403961
Current loss 0.06825154274702072


In [9]:
torch.save(model.state_dict(), './mnist/mnist.pth')

In [10]:
pretrained_model = Net()
pretrained_model.load_state_dict(torch.load('./mnist/mnist.pth'))

<All keys matched successfully>